In [9]:
# Install required packages
%pip install ultralytics
%pip install pillow
%pip install matplotlib
%pip install opencv-python
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
print(f"NumPy version: {np.__version__}")  # Should show 1.26.4 or similar

NumPy version: 1.26.4


In [11]:
import os, cv2, yaml, shutil
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Setup directories
root = Path.cwd()
dirs = {
    'train_img': root / "bird_dataset/images/train",
    'val_img': root / "bird_dataset/images/val", 
    'train_lbl': root / "bird_dataset/labels/train",
    'val_lbl': root / "bird_dataset/labels/val"
}
for d in dirs.values():
    d.mkdir(parents=True, exist_ok=True)

print(f"Setup complete! Dataset: {root / 'bird_dataset'}")


Setup complete! Dataset: /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset


In [12]:
source_path = "/Users/kevincorstorphine/Desktop/Bird_Soap/finetunepics"

def copy_images(src, dst):
    count = 0
    for f in os.listdir(src):
        if not os.path.isfile(os.path.join(src, f)):
            continue
        ext = f.lower().split('.')[-1]
        if ext in ['jpg', 'jpeg', 'png', 'bmp']:
            shutil.copy2(os.path.join(src, f), dst)
            count += 1
        elif ext in ['heic', 'heif']:
            try:
                Image.open(os.path.join(src, f)).save(
                    os.path.join(dst, f.rsplit('.', 1)[0] + '.jpg'), 'JPEG')
                count += 1
            except Exception as e:
                print(f"Failed to convert {f}: {e}")
    return count

copied = copy_images(source_path, dirs['train_img'])
print(f"Copied {copied} images")


Copied 10 images


In [13]:
model = YOLO('yolov8n.pt')

def annotate_images(img_dir, lbl_dir):
    count = 0
    for img_file in os.listdir(img_dir):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue
            
        img_path = os.path.join(img_dir, img_file)
        results = model(img_path)
        
        annotations = []
        if results[0].boxes is not None:
            img = cv2.imread(img_path)
            h, w = img.shape[:2]
            
            for box in results[0].boxes:
                if box.cls[0] == 14 and box.conf[0] > 0.5:  # Bird class
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    # Convert to YOLO format
                    cx, cy = (x1+x2)/(2*w), (y1+y2)/(2*h)
                    bw, bh = (x2-x1)/w, (y2-y1)/h
                    annotations.append(f"0 {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")
        
        if annotations:
            lbl_path = os.path.join(lbl_dir, img_file.rsplit('.', 1)[0] + '.txt')
            with open(lbl_path, 'w') as f:
                f.write('\n'.join(annotations))
            count += 1
    return count

# Generate annotations and split dataset
annotated = annotate_images(dirs['train_img'], dirs['train_lbl'])
print(f"Created {annotated} annotations")

# Split to validation (20%)
img_files = [f for f in os.listdir(dirs['train_img']) if f.endswith('.jpg')]
val_files = np.random.choice(img_files, int(len(img_files)*0.2), replace=False)

for f in val_files:
    shutil.move(dirs['train_img'] / f, dirs['val_img'] / f)
    lbl_f = f.rsplit('.', 1)[0] + '.txt'
    if (dirs['train_lbl'] / lbl_f).exists():
        shutil.move(dirs['train_lbl'] / lbl_f, dirs['val_lbl'] / lbl_f)

print(f"Split: {len(img_files) - len(val_files)} train, {len(val_files)} validation")



image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/train/IMG_1355.jpg: 640x480 1 traffic light, 1 bird, 82.5ms
Speed: 3.1ms preprocess, 82.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/train/IMG_1354.jpg: 640x480 (no detections), 95.1ms
Speed: 2.2ms preprocess, 95.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/train/IMG_1356.jpg: 640x480 2 birds, 218.8ms
Speed: 2.3ms preprocess, 218.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/train/IMG_1357.jpg: 640x480 (no detections), 214.9ms
Speed: 12.2ms preprocess, 214.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/train/IMG_1353.jpg: 640x480 1

In [14]:
# Create dataset config
config = {
    'path': str((root / "bird_dataset").absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'nc': 1,
    'names': ['bird']
}

config_path = root / "bird_dataset/dataset.yaml"
with open(config_path, 'w') as f:
    yaml.dump(config, f)

# Train model
model = YOLO('yolov8n.pt')
results = model.train(
    data=str(config_path),
    epochs=5,
    imgsz=640,
    batch=16,
    device='cpu',  # Change to 'cuda' if you have GPU
    project=str(root),
    name='bird_model',
    exist_ok=True
)

print("Training complete!")


Ultralytics 8.3.153 🚀 Python-3.11.6 torch-2.2.2 CPU (Intel Core(TM) i9-9980HK 2.40GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bird_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pers

train: Scanning /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/labels/train... 5 images, 3 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<00:00, 3608.78it/s]

train: New cache created: /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2584.9±1459.3 MB/s, size: 3916.3 KB)



val: Scanning /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/labels/val... 2 images, 1 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<00:00, 1250.54it/s]

val: New cache created: /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/labels/val.cache
Plotting labels to /Users/kevincorstorphine/Desktop/Bird_Soap/bird_model/labels.jpg... 


...

        5/5         0G     0.7024      3.701     0.9894         12        640: 100%|██████████| 1/1 [00:02<00:00,  2.87s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]

                   all          3          2    0.00222          1      0.497      0.497



5 epochs completed in 0.005 hours.
Optimizer stripped from /Users/kevincorstorphine/Desktop/Bird_Soap/bird_model/weights/last.pt, 6.2MB
Optimizer stripped from /Users/kevincorstorphine/Desktop/Bird_Soap/bird_model/weights/best.pt, 6.2MB

Validating /Users/kevincorstorphine/Desktop/Bird_Soap/bird_model/weights/best.pt...
Ultralytics 8.3.153 🚀 Python-3.11.6 torch-2.2.2 CPU (Intel Core(TM) i9-9980HK 2.40GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


                   all          3          2    0.00222          1      0.497      0.497
Speed: 1.4ms preprocess, 73.3ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to /Users/kevincorstorphine/Desktop/Bird_Soap/bird_model
Training complete!


In [15]:
# Load and test the trained model
model_path = root / 'bird_model/weights/best.pt'
if not model_path.exists():
    model_path = root / 'bird_model/weights/last.pt'

if model_path.exists():
    trained_model = YOLO(str(model_path))
    
    # Test on a sample image
    test_imgs = list(dirs['val_img'].glob('*.jpg')) or list(dirs['train_img'].glob('*.jpg'))
    if test_imgs:
        results = trained_model(str(test_imgs[0]))
        print(f"Test on {test_imgs[0].name}:")
        if results[0].boxes is not None:
            for box in results[0].boxes:
                if box.conf[0] > 0.5:
                    print(f"  Bird detected: {box.conf[0]:.3f} confidence")
        else:
            print("  No birds detected")
    
    # Export model
    export_path = root / "finetuned_bird_model.pt"
    shutil.copy2(model_path, export_path)
    print(f"\\nModel exported to: {export_path}")
    print(f"Use with: model = YOLO('{export_path}')")
else:
    print("No trained model found!")



image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/val/IMG_1355.jpg: 640x480 (no detections), 96.3ms
Speed: 2.2ms preprocess, 96.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)
Test on IMG_1355.jpg:
\nModel exported to: /Users/kevincorstorphine/Desktop/Bird_Soap/finetuned_bird_model.pt
Use with: model = YOLO('/Users/kevincorstorphine/Desktop/Bird_Soap/finetuned_bird_model.pt')


In [16]:
# Compare original vs fine-tuned model
if model_path.exists():
    original = YOLO('yolov8n.pt')
    trained = YOLO(str(model_path))
    
    test_imgs = list(dirs['val_img'].glob('*.jpg'))[:2]
    if not test_imgs:
        test_imgs = list(dirs['train_img'].glob('*.jpg'))[:2]
    
    fig, axes = plt.subplots(2, len(test_imgs), figsize=(12, 8))
    if len(test_imgs) == 1:
        axes = axes.reshape(2, 1)
    
    for i, img_path in enumerate(test_imgs):
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Original model
        orig_results = original(str(img_path))
        orig_img = img_rgb.copy()
        if orig_results[0].boxes is not None:
            for box in orig_results[0].boxes:
                if box.cls[0] == 14 and box.conf[0] > 0.5:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    cv2.rectangle(orig_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        # Fine-tuned model  
        ft_results = trained(str(img_path))
        ft_img = img_rgb.copy()
        if ft_results[0].boxes is not None:
            for box in ft_results[0].boxes:
                if box.conf[0] > 0.5:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    cv2.rectangle(ft_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        axes[0, i].imshow(orig_img)
        axes[0, i].set_title('Original Model')
        axes[0, i].axis('off')
        
        axes[1, i].imshow(ft_img)
        axes[1, i].set_title('Fine-tuned Model')  
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()




image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/val/IMG_1355.jpg: 640x480 1 traffic light, 1 bird, 122.9ms
Speed: 3.3ms preprocess, 122.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/val/IMG_1355.jpg: 640x480 (no detections), 80.9ms
Speed: 3.3ms preprocess, 80.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/val/IMG_1353.jpg: 640x480 1 bird, 114.1ms
Speed: 3.9ms preprocess, 114.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /Users/kevincorstorphine/Desktop/Bird_Soap/bird_dataset/images/val/IMG_1353.jpg: 640x480 (no detections), 88.2ms
Speed: 2.5ms preprocess, 88.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)


<Figure size 1200x800 with 4 Axes>